<a href="https://colab.research.google.com/github/viji-lab/aboutme/blob/main/8th_Jan_Dogs_vs_Cat_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf 
import cv2 
from google.colab.patches import cv2_imshow
import os

In [ ]:
import kaggle

In [ ]:
! mv kaggle.json /root/.kaggle

In [ ]:
! kaggle competitions download -c dogs-vs-cats

 99% 807M/812M [00:03<00:00, 245MB/s]
100% 812M/812M [00:03<00:00, 218MB/s]


In [ ]:
# unzip, rm, cd, ls are LINUX commands & are not Python functions! 
# We prepend a ! before Linux commands in jupyter 

In [ ]:
! unzip -q dogs-vs-cats.zip

In [ ]:
! rm sampleSubmission.csv
! rm test1.zip

In [ ]:
! unzip -q train.zip

In [ ]:
import os
len(os.listdir("train"))

25000

In [ ]:
os.listdir("train")[:10]

['cat.7493.jpg',
 'dog.8309.jpg',
 'dog.2144.jpg',
 'cat.12333.jpg',
 'dog.2457.jpg',
 'dog.9818.jpg',
 'dog.11983.jpg',
 'dog.7353.jpg',
 'dog.2016.jpg',
 'dog.1120.jpg']

In [ ]:
! ls train | grep dog | wc -l

12500


In [ ]:
! ls train | grep cat | wc -l

12500


In [ ]:
imageList = []
for imageName in os.listdir("train"):
  imageList.append(imageName.split(".")[0])

In [ ]:
len(imageList)

25000

In [ ]:
len([i for i in imageList if i == "dog"])

12500

In [ ]:
len([i for i in imageList if i == "cat"])

12500

In [ ]:
# Q) All the images have different shapes, our NN model accepts only same shape images... 
# How do we come to a common shape for images ?? 

In [ ]:
# (25000, 150, 150, 3) 

In [ ]:
# MNIST & Fashion-MNIST 
# Images are loaded into an array ? 

In [ ]:
images = [] # Image List 
labels = [] # Label List

for imageName in os.listdir("train"):
  image = cv2.imread("train/" + imageName)      # Read the image
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)# Converted the image from BGR to RGB
  image = cv2.resize(image, (150 , 150))        # Resized the image to 150x150
  images.append(image)

  if "dog" in imageName:                        # If image name contains dog
    labels.append("dog")
  elif "cat" in imageName:                      # If image name contains cat
    labels.append("cat")

In [ ]:
len(images)

25000

In [ ]:
len(labels)

25000

In [ ]:
imageArr = np.array(images)

In [ ]:
imageArr.shape        # Num of images x W x H x Channel

(25000, 150, 150, 3)

In [ ]:
labelArr = np.array(labels)

In [ ]:
labelArr.shape

(25000,)

In [ ]:
# Normalization of images
imageArr = imageArr / 255.0

In [ ]:
25000 * 150 * 150 * 3 

1687500000

In [ ]:
# 1.6B

In [ ]:
# Image Data Generator 
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [ ]:
idg = tf.keras.preprocessing.image.ImageDataGenerator()

In [ ]:
# idg.flow_from_dataframe       # Read data through a DataFrame
# idg.flow_from_directory       # Read the images directory from Folders

In [ ]:
?idg.flow_from_directory

In [ ]:
#   mainFolder    # directory
#     | - cat1    # sub directory
#     | - cat2

In [ ]:
#   trainimages 
#   | - cat
#   | - dog

In [ ]:
os.mkdir("trainimages")
os.mkdir("trainimages/cat")
os.mkdir("trainimages/dog")

In [ ]:
import shutil
from shutil import copy

In [ ]:
# copy(source, destination)

In [ ]:
source = "train/"
destCat = "trainimages/cat/"
destDog = "trainimages/dog/"
for image in os.listdir("train"):
  if "dog" in image:
    copy(source + image, destDog + image)
  else:
    copy(source + image, destCat + image)

In [ ]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)
trainGen = idg.flow_from_directory("trainimages", 
                                   target_size=(150, 150), 
                                   batch_size=32)

Found 25000 images belonging to 2 classes.


In [ ]:
# del model

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input((150, 150, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(120, activation = tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(2, activation = tf.keras.activations.softmax))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 67500)             0         
                                                                 
 dense_2 (Dense)             (None, 120)               8100120   
                                                                 
 dense_3 (Dense)             (None, 2)                 242       
                                                                 
Total params: 8,100,362
Trainable params: 8,100,362
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(), 
              loss = tf.keras.losses.categorical_crossentropy, 
              metrics = ["acc"])

In [ ]:
model.fit(trainGen, batch_size=32, epochs=10)

Epoch 1/10
782/782 [==============================] - 113s 143ms/step - loss: 0.6937 - acc: 0.5718
Epoch 2/10
782/782 [==============================] - 119s 152ms/step - loss: 0.6580 - acc: 0.6092
Epoch 3/10
782/782 [==============================] - 114s 145ms/step - loss: 0.6490 - acc: 0.6204
Epoch 4/10
782/782 [==============================] - 111s 141ms/step - loss: 0.6386 - acc: 0.6332
Epoch 5/10
782/782 [==============================] - 111s 142ms/step - loss: 0.6340 - acc: 0.6361
Epoch 6/10
782/782 [==============================] - 108s 139ms/step - loss: 0.6282 - acc: 0.6422
Epoch 7/10
782/782 [==============================] - 109s 140ms/step - loss: 0.6221 - acc: 0.6492
Epoch 8/10
782/782 [==============================] - 107s 137ms/step - loss: 0.6170 - acc: 0.6545
Epoch 9/10
782/782 [==============================] - 108s 138ms/step - loss: 0.6115 - acc: 0.6604
Epoch 10/10
782/782 [==============================] - 108s 138ms/step - loss: 0.6074 - acc: 0.6646
